# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LrExercise').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [5]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [7]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [8]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [13]:
indexer = StringIndexer(inputCol = 'Cruise_line', outputCol='Cruise_index').fit(df)
df2 = indexer.transform(df)
df2.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|        16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|        16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|         1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|         1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|         1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|         1.0|
|    Elati

In [14]:
assembler = VectorAssembler(inputCols=['Tonnage', 'passengers', 'length', 
                                       'cabins', 'passenger_density','Cruise_index'], outputCol='features')

In [16]:
output = assembler.transform(df2)

In [18]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[30.2769999999999...|
|[30.2769999999999...|
|[47.262,14.86,7.2...|
|[110.0,29.74,9.53...|
|[101.353,26.42,8....|
|[70.367,20.52,8.5...|
|[70.367,20.52,8.5...|
|[70.367,20.56,8.5...|
|[70.367,20.52,8.5...|
|[110.238999999999...|
|[110.0,29.74,9.51...|
|[46.052,14.52,7.2...|
|[70.367,20.52,8.5...|
|[70.367,20.52,8.5...|
|[86.0,21.24,9.63,...|
|[110.0,29.74,9.51...|
|[88.5,21.24,9.63,...|
|[70.367,20.52,8.5...|
|[88.5,21.24,9.63,...|
|[70.367,20.52,8.5...|
+--------------------+
only showing top 20 rows



In [19]:
finalData = output.select('features','crew')

In [21]:
finalData.head(1)

[Row(features=DenseVector([30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), crew=3.55)]

In [23]:
trainData, testData = finalData.randomSplit([0.7,0.3])

In [24]:
trainData.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              118|
|   mean|7.934576271186442|
| stddev|3.379347948045901|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [25]:
testData.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               40|
|   mean|7.380000000000003|
| stddev|3.862232647788197|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [26]:
lr = LinearRegression(featuresCol='features',labelCol='crew')

In [28]:
lrModel = lr.fit(trainData)

In [29]:
result = lrModel.evaluate(testData)

In [30]:
result.rootMeanSquaredError

0.76024654568781

In [31]:
result.r2

0.9602600392308094